# Migration Guide

## Introduction

This jupyter notebook demonstrates how to adapt a [pycycling](https://github.com/zacharyedwardbull/pycycling) device into Bleak-FSM.

For this example we will use . Sterzo is a bluetooth steering angle sensor that goes under the front wheel of a bicycle mounted on an indoor trainer. 

The implementation of Sterzo is at [pycycling/sterzo.py](https://github.com/zacharyedwardbull/pycycling/blob/master/pycycling/sterzo.py) but we only need to look at [examples/sterzo_examples.py](https://github.com/zacharyedwardbull/pycycling/blob/master/examples/sterzo_example.py).

For your convenience, here is `sterzo_example.py`:

```python

import asyncio
from bleak import BleakClient

from pycycling.sterzo import Sterzo


async def run(address):
    async with BleakClient(address) as client:
        def steering_handler(steering_angle):
            print(steering_angle)

        await client.is_connected()
        sterzo = Sterzo(client)
        sterzo.set_steering_measurement_callback(steering_handler)
        await sterzo.enable_steering_measurement_notifications()
        await asyncio.sleep(60)

if __name__ == "__main__":
    import os

    os.environ["PYTHONASYNCIODEBUG"] = str(1)

    device_address = "36A444C9-2A18-4B6B-B671-E0A8D3DADB1D"
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run(device_address))

```

## Setup

To begin with, we'll import the same modules plus Bleak-FSM

In [ ]:
import asyncio
from pycycling.sterzo import Sterzo

from bleak_fsm import BleakModel


In [ ]:
model = BleakModel() # This represents a given bluetooth adapter. Let's assume there's one on the system.

By default, `model.wrap` is a function that does nothing and just returns the input.
That's fine if we want to use the `client`, which is an instance of `BleakClient` directly.

However, we want to take advantage of the work that went into creating the `Sterzo` implementation.
Therefore we need to modify the `model.wrap` function as follows:

In [ ]:
model.wrap = lambda client: Sterzo(client) # Wrap the model with the Sterzo class

If you look at [line 54 of sterzo.py](https://github.com/zacharyedwardbull/pycycling/blob/a5998eb191edb1b2f2591d26b06f442a4f1ccff3/pycycling/sterzo.py#L54), you can see that the steering measurement callback is called with one positional argument, called `steering_value`. 
The steering measurement callback is set according through `set_steering_measurement_callback()` ([line 46-47](https://github.com/zacharyedwardbull/pycycling/blob/a5998eb191edb1b2f2591d26b06f442a4f1ccff3/pycycling/sterzo.py#L46-L47))

We are responsible for creating and passing in a callback function that will become `self._steering_measurement_callback`

In [ ]:
def handle_sterzo_measurement(value):
    print(f"Sterzo angle: {value}")

In [ ]:
model.set_measurement_handler = lambda client: client.set_steering_measurement_callback(handle_sterzo_measurement)

Next, we set the `enable_notifications` and `disable_notifications` functions to their corresponding functions in [examples/sterzo_examples.py](https://github.com/zacharyedwardbull/pycycling/blob/master/examples/sterzo_example.py)

In [ ]:
model.enable_notifications = lambda client: client.enable_steering_measurement_notifications()
model.disable_notifications = lambda client: client.disable_steering_measurement_notifications()

## Using our new Bleak-FSM interface to Sterzo

In [ ]:
model.state

In [ ]:
await BleakModel.start_scan()

In [ ]:
await asyncio.sleep(3)

In [ ]:
await BleakModel.stop_scan()

In [ ]:
model.state

In [ ]:
len(BleakModel.bt_devices)

Since Elite Sterzo is not registered on any official Bluetooth spec, the service UUID is chosen arbitrary by them. This service UUID is found by Googling.

In [ ]:
STERZO_SERVICE_UUID = '347b0001-7635-408b-8918-8ff3949ce592'

sterzo_address = None
# Look for Sterzo device based on whether it advertises the Sterzo service
for address, (ble_device, advertisement_data) in BleakModel.bt_devices.items():
    if STERZO_SERVICE_UUID in advertisement_data.service_uuids:
        sterzo_address = address
        print(f"Found Sterzo at {sterzo_address}")
        break

In [ ]:
if sterzo_address is not None:
    await model.set_target(sterzo_address)
else:
    raise Exception("No sterzo found. Make sure it's on and the system bluetooth is on.")

In [ ]:
model.state

In [ ]:
success = await model.connect()
if not success:
    raise Exception("Could not connect to Sterzo")

In [ ]:
model.state

In [ ]:
await model.stream()

In [ ]:
await asyncio.sleep(10)

In [ ]:
await model.clean_up()